# Network Setting 1

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
import tensorflow as tf


train = pd.read_csv('train_new.csv')
test = pd.read_csv('test_new.csv')

train = pd.read_csv('train_new.csv')
train = train.fillna(0)

test = pd.read_csv('test_new.csv')
test = test.fillna(0)

test_id = test['Id']
test = test.drop(columns = ['Id', 'text', 'reply_to_screen_name', 'hashtags', 'clean_tweet'], axis = 1)

## Defining input and target
X = train.drop(columns = ['text', 'reply_to_screen_name', 'hashtags', 'clean_tweet', 'country'], axis = 1)
Y = train['country']
Y = np.where(Y == 'us', 0, 
             np.where(Y == 'uk', 1, 
                      np.where(Y == 'canada', 2, 
                               np.where(Y == 'australia', 3,
                                        np.where(Y == 'ireland', 4, 5)))))

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.2)

## Scaling the data 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
test = scaler.fit_transform(test)

## Defining model 
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10, input_dim =  10, activation = 'relu'),
#         tf.keras.layers.Dense(10, input_dim =  15, activation = 'relu'),
        tf.keras.layers.Dense(6, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Fitting model 
model.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 6), epochs = 20, batch_size = 32, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 6)))

## Predicting on test
nn_pred = model.predict(test)
nn_pred = np.argmax(nn_pred, axis = 1)

## Defining data to be exported
data_out = pd.DataFrame({'Id': test_id, 'Category': nn_pred})
data_out['Category'] = np.where(data_out['Category'] == 0, 'us',
                                np.where(data_out['Category'] == 1, 'uk',
                                         np.where(data_out['Category'] == 2, 'canada',
                                                  np.where(data_out['Category'] == 3, 'australia',
                                                           np.where(data_out['Category'] == 4, 'ireland', 'new_zealand')))))
data_out.to_csv('nn_submission_md6.csv', index = False)

Epoch 1/20
1500/1500 [==============================] - 10s 5ms/step - loss: 1.4811 - accuracy: 0.2998 - val_loss: 1.3646 - val_accuracy: 0.3114
Epoch 2/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.3520 - accuracy: 0.3173 - val_loss: 1.3488 - val_accuracy: 0.3137
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.3451 - accuracy: 0.3179 - val_loss: 1.3451 - val_accuracy: 0.3168
Epoch 4/20
1500/1500 [==============================] - 7s 4ms/step - loss: 1.3427 - accuracy: 0.3182 - val_loss: 1.3436 - val_accuracy: 0.3168
Epoch 5/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.3414 - accuracy: 0.3205 - val_loss: 1.3426 - val_accuracy: 0.3169
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.3406 - accuracy: 0.3226 - val_loss: 1.3421 - val_accuracy: 0.3228
Epoch 7/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.3402 - accuracy: 0.3192 - val_loss: 1.3418 - val_accuracy

In [7]:
nn_pred = model.predict(X_test)
nn_pred = np.argmax(nn_pred, axis = 1)

## Defining data to be exported
data_out = pd.DataFrame({'Actual': Y_test, 'Pred': nn_pred})
data_out['Actual'] = np.where(data_out['Actual'] == 0, 'us',
                              np.where(data_out['Actual'] == 1, 'uk',
                                       np.where(data_out['Actual'] == 2, 'canada',
                                                np.where(data_out['Actual'] == 3, 'australia',
                                                         np.where(data_out['Actual'] == 4, 'ireland', 'new_zealand')))))

data_out['Pred'] = np.where(data_out['Pred'] == 0, 'us',
                            np.where(data_out['Pred'] == 1, 'uk',
                                     np.where(data_out['Pred'] == 2, 'canada',
                                              np.where(data_out['Pred'] == 3, 'australia',
                                                       np.where(data_out['Pred'] == 4, 'ireland', 'new_zealand')))))
data_out.head()

,Actual,Pred
0,australia,new_zealand
1,australia,us
2,new_zealand,ireland
3,uk,us
4,australia,ireland


In [12]:
from sklearn.metrics import classification_report

target_names = ['us', 'uk', 'canada', 'australia', 'ireland', 'new_zealand']
print(classification_report(data_out['Actual'], data_out['Pred'], target_names = target_names))

              precision    recall  f1-score   support

          us       0.28      0.00      0.00     32133
          uk       0.32      0.03      0.06     31860
      canada       0.37      0.72      0.49     32102
   australia       0.40      0.19      0.26     31914
     ireland       0.29      0.64      0.40     31981
 new_zealand       0.31      0.38      0.34     32010

    accuracy                           0.33    192000
   macro avg       0.33      0.33      0.26    192000
weighted avg       0.33      0.33      0.26    192000

